<a href="https://colab.research.google.com/github/RUDHRAMURTHI/My-codes/blob/master/MachineHack_UnitPrice_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.listdir()

['.config', 'test_cleaned.csv', 'train_cleaned.csv', 'sample_data']

In [ ]:
data_train = pd.read_csv('train_cleaned.csv')

In [ ]:
data_test = pd.read_csv('test_cleaned.csv')

In [ ]:
data_train.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday
0,6141,1583,144,3,3.75,14056.0,35,5,6,16,18,4
1,6349,1300,3682,6,1.95,13098.0,35,5,11,7,19,2
2,16783,2178,1939,4,5.95,15044.0,35,11,20,13,46,6
3,16971,2115,2983,1,0.83,15525.0,35,11,22,12,47,1
4,6080,1210,2886,12,1.65,13952.0,35,5,6,9,18,4


In [ ]:
data_test.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday
0,3139,1709,1079,3,16161.0,35,2,22,15,8,1
1,617,510,3457,1,17341.0,35,12,8,12,49,2
2,14653,604,694,36,15158.0,35,10,25,13,43,1
3,8634,1478,3473,2,16033.0,35,6,27,12,26,0
4,15546,3216,871,1,15351.0,35,11,6,16,44,6


In [ ]:
X = data_train.drop(["UnitPrice"],axis=1)

In [ ]:
X = pd.get_dummies(X)

In [ ]:
columns = X.columns.to_list()
columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'CustomerID',
 'Country',
 'InvoiceDate_Month',
 'InvoiceDate_Day',
 'InvoiceDate_hour',
 'InvoiceDate_week',
 'InvoiceDate_weekday']

In [ ]:
y = data_train["UnitPrice"]

In [ ]:
X_sub = data_test

In [ ]:
X_sub = pd.get_dummies(X_sub)

In [ ]:
X.shape

(284773, 11)

In [ ]:
X_sub.shape

(122049, 11)

In [ ]:
y.shape

(284773,)

### Scaling the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mm_scaler = MinMaxScaler()

In [ ]:
X = mm_scaler.fit_transform(X)

In [ ]:
X_sub = mm_scaler.transform(X_sub)

In [ ]:
X.shape

(284773, 11)

In [ ]:
X_sub.shape

(122049, 11)

In [ ]:
train_data = pd.DataFrame(X, columns=columns)
train_data.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday
0,0.276771,0.429813,0.036970,0.310273,0.287710,0.972222,0.363636,0.166667,0.714286,0.34,0.666667
1,0.286146,0.352973,0.945315,0.310754,0.126431,0.972222,0.363636,0.333333,0.071429,0.36,0.333333
2,0.756400,0.591366,0.497818,0.310433,0.454040,0.972222,0.909091,0.633333,0.500000,0.90,1.000000
3,0.764873,0.574260,0.765854,0.309952,0.535017,0.972222,0.909091,0.700000,0.428571,0.92,0.166667
4,0.274022,0.328537,0.740950,0.311717,0.270202,0.972222,0.363636,0.166667,0.214286,0.34,0.666667


In [ ]:
pred_data = pd.DataFrame(X_sub, columns=columns)
pred_data.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday
0,0.141473,0.464024,0.277022,0.310273,0.642088,0.972222,0.090909,0.700000,0.642857,0.14,0.166667
1,0.027808,0.138474,0.887548,0.309952,0.840741,0.972222,1.000000,0.233333,0.428571,0.96,0.333333
2,0.660402,0.163997,0.178177,0.315570,0.473232,0.972222,0.818182,0.800000,0.500000,0.84,0.166667
3,0.389129,0.401303,0.891656,0.310112,0.620539,0.972222,0.454545,0.866667,0.428571,0.50,0.000000
4,0.700649,0.873201,0.223620,0.309952,0.505724,0.972222,0.909091,0.166667,0.714286,0.86,1.000000


In [ ]:
train_data['target'] = y
train_data.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday,target
0,0.276771,0.429813,0.036970,0.310273,0.287710,0.972222,0.363636,0.166667,0.714286,0.34,0.666667,3.75
1,0.286146,0.352973,0.945315,0.310754,0.126431,0.972222,0.363636,0.333333,0.071429,0.36,0.333333,1.95
2,0.756400,0.591366,0.497818,0.310433,0.454040,0.972222,0.909091,0.633333,0.500000,0.90,1.000000,5.95
3,0.764873,0.574260,0.765854,0.309952,0.535017,0.972222,0.909091,0.700000,0.428571,0.92,0.166667,0.83
4,0.274022,0.328537,0.740950,0.311717,0.270202,0.972222,0.363636,0.166667,0.214286,0.34,0.666667,1.65


In [ ]:
train, test = train_test_split(train_data, test_size=0.2)
print(len(train), 'train examples')
print(len(pred_data), 'prediction examples')
print(len(test), 'test examples')

227818 train examples
122049 prediction examples
56955 test examples


### Create an input pipeline using tf.data

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(test, shuffle=False)

In [60]:
train_ds.take(2)

<TakeDataset shapes: ({InvoiceNo: (None,), StockCode: (None,), Description: (None,), Quantity: (None,), CustomerID: (None,), Country: (None,), InvoiceDate_Month: (None,), InvoiceDate_Day: (None,), InvoiceDate_hour: (None,), InvoiceDate_week: (None,), InvoiceDate_weekday: (None,)}, (None,)), types: ({InvoiceNo: tf.float64, StockCode: tf.float64, Description: tf.float64, Quantity: tf.float64, CustomerID: tf.float64, Country: tf.float64, InvoiceDate_Month: tf.float64, InvoiceDate_Day: tf.float64, InvoiceDate_hour: tf.float64, InvoiceDate_week: tf.float64, InvoiceDate_weekday: tf.float64}, tf.float64)>

In [ ]:
InvoiceNo = keras.Input(shape=(1,), name="InvoiceNo")
StockCode = keras.Input(shape=(1,), name="StockCode")
Description = keras.Input(shape=(1,), name="Description")
Quantity = keras.Input(shape=(1,), name="Quantity")
CustomerID = keras.Input(shape=(1,), name="CustomerID")
Country = keras.Input(shape=(1,), name="Country")
InvoiceDate_Month = keras.Input(shape=(1,), name="InvoiceDate_Month")
InvoiceDate_Day = keras.Input(shape=(1,), name="InvoiceDate_Day")
InvoiceDate_hour = keras.Input(shape=(1,), name="InvoiceDate_hour")
InvoiceDate_week = keras.Input(shape=(1,), name="InvoiceDate_week")
InvoiceDate_weekday = keras.Input(shape=(1,), name="InvoiceDate_weekday")

In [ ]:
all_features = layers.concatenate(
    [
        InvoiceNo,
        StockCode,
        Description,
        Quantity,
        CustomerID,
        Country,
        InvoiceDate_Month,
        InvoiceDate_Day,
        InvoiceDate_hour,
        InvoiceDate_week,
        InvoiceDate_weekday
    ]
)

In [ ]:
all_inputs= [
        InvoiceNo,
        StockCode,
        Description,
        Quantity,
        CustomerID,
        Country,
        InvoiceDate_Month,
        InvoiceDate_Day,
        InvoiceDate_hour,
        InvoiceDate_week,
        InvoiceDate_weekday
    ]

In [ ]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
x = layers.Dense(64, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)

model.compile(optimizer='Adadelta', loss='mean_squared_error', metrics=tf.keras.metrics.MeanSquaredError())

In [ ]:
history = model.fit(train_ds, batch_size=50, epochs=10, validation_data=val_ds)

Epoch 1/10
7120/7120 [==============================] - 32s 5ms/step - loss: 620.3503 - mean_squared_error: 620.3503 - val_loss: 466.3860 - val_mean_squared_error: 466.3860
Epoch 2/10
7120/7120 [==============================] - 32s 4ms/step - loss: 620.1402 - mean_squared_error: 620.1402 - val_loss: 466.2043 - val_mean_squared_error: 466.2043
Epoch 3/10
7120/7120 [==============================] - 31s 4ms/step - loss: 619.9677 - mean_squared_error: 619.9677 - val_loss: 466.0290 - val_mean_squared_error: 466.0290
Epoch 4/10
7120/7120 [==============================] - 31s 4ms/step - loss: 619.7841 - mean_squared_error: 619.7841 - val_loss: 465.8633 - val_mean_squared_error: 465.8633
Epoch 5/10
7120/7120 [==============================] - 31s 4ms/step - loss: 619.6262 - mean_squared_error: 619.6262 - val_loss: 465.7066 - val_mean_squared_error: 465.7066
Epoch 6/10
7120/7120 [==============================] - 33s 5ms/step - loss: 619.4671 - mean_squared_error: 619.4671 - val_loss: 465.55

In [59]:
pred_data.head()

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,InvoiceDate_Month,InvoiceDate_Day,InvoiceDate_hour,InvoiceDate_week,InvoiceDate_weekday
0,0.141473,0.464024,0.277022,0.310273,0.642088,0.972222,0.090909,0.700000,0.642857,0.14,0.166667
1,0.027808,0.138474,0.887548,0.309952,0.840741,0.972222,1.000000,0.233333,0.428571,0.96,0.333333
2,0.660402,0.163997,0.178177,0.315570,0.473232,0.972222,0.818182,0.800000,0.500000,0.84,0.166667
3,0.389129,0.401303,0.891656,0.310112,0.620539,0.972222,0.454545,0.866667,0.428571,0.50,0.000000
4,0.700649,0.873201,0.223620,0.309952,0.505724,0.972222,0.909091,0.166667,0.714286,0.86,1.000000


In [64]:
sample = {
        "InvoiceNo": pred_data.InvoiceNo.to_list(),
        "StockCode": pred_data.StockCode.to_list(),
        "Description": pred_data.Description.to_list(),
        "Quantity": pred_data.Quantity.to_list(),
        "CustomerID": pred_data.CustomerID.to_list(),
        "Country": pred_data.Country.to_list(),
        "InvoiceDate_Month": pred_data.InvoiceDate_Month.to_list(),
        "InvoiceDate_Day": pred_data.InvoiceDate_Day.to_list(),
        "InvoiceDate_hour": pred_data.InvoiceDate_hour.to_list(),
        "InvoiceDate_week": pred_data.InvoiceDate_week.to_list(),
        "InvoiceDate_weekday": pred_data.InvoiceDate_weekday.to_list(),
}

In [66]:
#input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(sample)

ValueError: ignored